In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

from IPython.display import display, JSON
import plotly.graph_objects as go

from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_france import FranceTaxBenefitSystem
import openfisca_interactive

In [3]:
tax_benefit_system = FranceTaxBenefitSystem()
decomposition = tax_benefit_system.load_decomposition()
tax_benefit_system.add_variables_from_decomposition_tree(decomposition, update=True)

In [4]:
bars = openfisca_interactive.get_bars(decomposition)

In [5]:
def build_test_case(salaire_min, salaire_max, salaire_count, period):
    return {
    "individus": {
        "Michel": {
            'date_naissance': {'ETERNITY': '1980-01-01'},
        },
    },
    "familles": {
        "famille_1": {
            "parents": ["Michel"]
        }
    },
    "foyers_fiscaux": {
        "foyer_fiscal_1": {
            "declarants": ["Michel"],
        },
    },
    "menages": {
        "menage_1": {
            "personne_de_reference": ["Michel"],
        }},
    "axes": [[
        {
            "name": 'salaire_de_base',
            "count": salaire_count,
            "min": salaire_min,
            "max": salaire_max,
            "period": period,
        },
    ]],
}

In [11]:
period = 2019
salaire_min = value=0
salaire_max = 100_000
salaire_count = 101
salaire_index = 20

test_case = build_test_case(salaire_min=salaire_min, salaire_max=salaire_max, salaire_count=salaire_count, period=period)
simulation_builder = SimulationBuilder()
simulation = simulation_builder.build_from_entities(tax_benefit_system, test_case)
salaire_de_base_arr = simulation.calculate_add("salaire_de_base", period=period)
results = openfisca_interactive.calculate_bars(bars, simulation, period=period)

In [12]:
displayed_bars = list(openfisca_interactive.iter_displayed_bars(
    bars, results, x_variable_arr=salaire_de_base_arr, x_variable=salaire_de_base_arr[salaire_index],
    include_subtotals=False, include_zero=False))

In [13]:
BarType = openfisca_interactive.BarType

waterfall = go.Waterfall()
waterfall.x = [bar.short_name or bar.variable_name for bar in displayed_bars]
waterfall.y = [
    bar.value if bar.type == BarType.VALUE else 0
    for bar in displayed_bars
]
waterfall.measure = [
    "relative" if bar.type == BarType.VALUE else "total"
    for bar in displayed_bars
]
waterfall.text = [
    ("{:+.0f}" if bar.type == BarType.VALUE else "{:.0f}").format(bar.value)
    for bar in displayed_bars
]

layout = go.Layout(title=f"Décomposition du revenu disponible ({period})")
go.FigureWidget(data=[waterfall], layout=layout)

FigureWidget({
    'data': [{'measure': [relative, relative, relative, relative, relative,
                   …